In [ ]:
# hide
# dont_test
# default_exp handlers
%load_ext nb_black

<IPython.core.display.Javascript object>

# ASGIHandlers

In [ ]:
# export

from asgiref.sync import sync_to_async
from django.core.handlers.asgi import ASGIHandler


class AsyncFileASGIHandler(ASGIHandler):
    async def send_response(self, response, send):
        is_async_fileresponse = getattr(response, "is_async_fileresponse", False)
        if is_async_fileresponse:
            return await response.stream(send)
        else:
            return await super().send_response(response, send)

<IPython.core.display.Javascript object>

## Tests

In [ ]:
# hide
import django

from django.conf import settings

try:
    settings.configure()
except RuntimeError:
    pass

<IPython.core.display.Javascript object>

In [ ]:
class Response:
    cookies = {}
    streaming = True
    status_code = 200
    stream_called = False

    def __init__(self, content, is_async_fileresponse=None):
        self.index = -1
        self.contents = [content]
        if is_async_fileresponse is not None:
            self.is_async_fileresponse = is_async_fileresponse

    async def stream(self, send):
        self.stream_called = True
        for content in self.contents:
            await send(content)

    def __iter__(self):
        return self

    def __next__(self):
        self.index += 1
        try:
            return self.contents[self.index]
        except IndexError:
            raise StopIteration

    def items(self):
        return []

    def close(self):
        pass


received_send = []


async def send(data):
    received_send.append(data)

<IPython.core.display.Javascript object>

### Happy Path

In [ ]:
app = AsyncFileASGIHandler()
content = "<h1>Content!</h1>"
response = Response(content, is_async_fileresponse=True)
received_send = []
await app.send_response(response, send)
returned_content = received_send[0]
assert response.stream_called
assert content == returned_content

<IPython.core.display.Javascript object>

### Response Without is_async_fileresponse attribute

In [ ]:
app = AsyncFileASGIHandler()
content = "<h1>Content!</h1>"
response = Response(content)
received_send = []
await app.send_response(response, send)
returned_content = received_send[1]["body"]
assert not response.stream_called
assert content == returned_content

<IPython.core.display.Javascript object>